# How To Difference Two Images

_Contact: Leanne Guy, Colin Slater, Phil Marshall_

_Last edited: 2018-05-30_

_Runs with Stack release: Weekly 2018-21_

In this notebook we run the `ImageDifferenceTask` on two visit images, and inspect the result.

## Setups

In [ ]:
# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afw_display

# Firefly client imports
from firefly_client import FireflyClient

# Standard libraries in support of Firefly display
from urllib.parse import urlparse, urlunparse, ParseResult
from IPython.display import IFrame, display, Markdown
import os

## Finding Two Images

We'll use two single sensor images from the HSC WIDE dataset, taken close together in time and at the same point on the sky. 

Data is in `/datasets/hsc/repo/rerun/DM-10404/WIDE/`, cheating can be achieved via the sqlite3 database in `/datasets/hsc/repo/registry.sqlite3`: 
```
    select * from raw limit 10
```

In [ ]:
data_dir = '/datasets/hsc/repo/rerun/DM-10404/WIDE/'

In [ ]:
# Create a butler from this data directory
from lsst.daf.persistence import Butler
butler = Butler(data_dir)
#print(butler.getUri)
# get all the visits over the SSP_WIDE survey
visitPointings = butler.queryMetadata('calexp',('visit', 'pointing'),{'field':'SSP_WIDE'})
#print(visitPointings)

In [ ]:
# Lets try to pull up 2 images from different fields of the same pointing
# 374, 376, pointing :  814
dataId1 = {'visit': 374, 'ccd':42}
dataId2 = {'visit': 376, 'ccd':42}

In [ ]:
calexp1 = butler.get('calexp', **dataId1)
calexp2 = butler.get('calexp', **dataId2)

In [ ]:
# Create a display in a new browser tab:
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'

In [ ]:
# Uncomment the lines below, to get a display inside this notebook:

#ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
#IFrame(ff,800,600)

In [ ]:
afw_display.setDefaultBackend('firefly')
afw_display_1 = afw_display.getDisplay(frame=1, 
                                    name=my_channel)
afw_display_2 = afw_display.getDisplay(frame=2, 
                                    name=my_channel)

In [ ]:
afw_display_1.mtv(calexp1, "Calexp %d" % dataId1['visit'])
afw_display_2.mtv(calexp2, "Calexp %d" % dataId2['visit'])

In [ ]:
# C1 date
c1VisitInfo = calexp1.getInfo().getVisitInfo()
c1Date = c1VisitInfo.getDate()

# C2
c2VisitInfo = calexp2.getInfo().getVisitInfo()
c2Date = c2VisitInfo.getDate()

# Time difference 
timediff = c2Date.toPython() -c1Date.toPython()
timediff.total_seconds()

# Try to identify visits with the same exposure time 
for visit,pointing in visitPointings[:10] : 
    dataId = {'visit': visit, 'ccd':42}
    calexp = butler.get('calexp', **dataId)
    visitInfo = calexp.getInfo().getVisitInfo()
    print(visit, ",", visitInfo.getExposureTime())


Looks like `374` and `376` will do just fine - and `376`, with its longer exposure time, will make a plausible model template image.

In [ ]:
image_dataId = {'visit': 374, 'ccd':42}
template_dataId = {'visit': 376, 'ccd':42}

## Image Differencing

The relevant pipeline task is [`ImageDifferenceTask`](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/imageDifference.py#L214), wrapped in the following command line task.

In [ ]:
! imageDifference.py

The command line task does not complete, in our simple visit-visit example. Instead, let's call the python commands directly. (The command line that ought to run is given below, commented out, for completeness.)

In [ ]:
# ! imageDifference.py $data_dir --output 'diffim-output' --id visit=image_dataId ccd=42 --templateId visit=template_dataId --configfile diffim-config.py

In [ ]:
# Set up an ImageDifferenceTask object:
from lsst.pipe.tasks.imageDifference import ImageDifferenceTask, ImageDifferenceConfig 
from lsst.ip.diffim.getTemplate import GetCalexpAsTemplateTask

config = ImageDifferenceConfig()
config.getTemplate.retarget(GetCalexpAsTemplateTask)
config.doWriteSources=False
config.doWriteSubtractedExp=False
imageDifference = ImageDifferenceTask(butler=butler, config=config)

# Prepare the science image, and the template to difference it with:
image = butler.dataRef('calexp', dataId=image_dataId)
# template = butler.dataRef('calexp', dataId=template_dataId) 
# NB: the template is not a Butler dataref objects, but instead a data ID:
template = template_dataId

# Take the difference:
diffedImage = imageDifference.run(image, templateIdList=[template])

In [ ]:
# To help debug the above commands, try using the python debugger:
# %pdb
# (This is a toggle.)

Let's just print the resulting structure, to see what we have: 

In [ ]:
diffedImage

In [ ]:
# Plot the differenced image in the firefly window.
afw_display_2.mtv(diffedImage.subtractedExposure)

The layers can be adjusted in the firefly display, to see the DIAsources.